## Background code

### PNG encryption with RSA 

In [341]:
import os
import zlib
import struct
import random
import datetime
from classes.rsa import RSA 
from classes.png import PNG 

### Helper functions

In [342]:
def writeEncrypted(data, name, timestamp):
    filename_no_ext = name[:-4]
    new_name = f"{timestamp}_{filename_no_ext}_encrypted.png"
    with open(new_name, 'wb') as new_file:
        for chunk in data:
            new_file.write(chunk)
    return new_name

def writeDecrypted(data, name):
    filename_no_ext = name[:-14]
    new_name = f"{filename_no_ext}_decrypted.png"
    with open(new_name, 'wb') as new_file:
        for chunk in data:
            new_file.write(chunk)

def saveKeys(keys, name, timestamp):
    filename_no_ext = name[:-4]
    new_name = f"{timestamp}_{filename_no_ext}_keys.txt"
    pubkey = keys[0]
    privkey = keys[1]
    with open(new_name, 'w') as new_file:
        new_file.write(f"Public key:\n{pubkey[0]}\n{pubkey[1]}\n")
        new_file.write(f"Private key:\n{privkey[0]}\n{privkey[1]}\n")
    return new_name

def loadKeys(filename):
    with open(filename, 'r') as keyfile:
        lines = keyfile.readlines()
        n_pub = int(lines[1])
        e_pub = int(lines[2])
        d_priv = int(lines[4])
        e_priv = int(lines[5])

        pubkey = (n_pub, e_pub)
        privkey = (d_priv, e_priv)

        return (pubkey, privkey)  

## Encryption & Decryption demos

#### Keypair generation

In [343]:
keypair = RSA.generateKeypair()
print(keypair)

((1995424812039771229371121401557, 65537), (1995424812039771229371121401557, 1172251563061218865920577060001))


#### Path to image

In [344]:
filename = "test.png"
img_path = f"media/{filename}"

### Szyfrowanie danych bez dekompresji danych z pliku

##### Encryption

In [345]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

encrypted_idat_chunks = []
for idat in idat_chunks:
    idat_data = idat[8:-4]

    try:
        decompressed_data = zlib.decompress(idat_data)
    except zlib.error:
        decompressed_data = idat_data

    encrypted_uncompressed = RSA.encryptData(decompressed_data, pubkey)
    encrypted_compressed = zlib.compress(encrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(encrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_compressed))
    new_chunk = new_len + idat_type + encrypted_compressed + new_crc

    encrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

timestamp = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
encrypt_img_path = writeEncrypted(all_chunks, png_name, timestamp)
keyfile = saveKeys(keypair, png_name, timestamp)

##### Decryption

In [346]:

keys = loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    encrypted_uncompressed  = zlib.decompress(chunk[8:-4])
    decrypted_uncompressed  = RSA.decryptData(encrypted_uncompressed, privkey)
    decrypted_compressed = zlib.compress(decrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(decrypted_compressed).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_compressed))
    new_chunk = new_len + idat_type + decrypted_compressed + new_crc

    decrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

writeDecrypted(all_chunks, png_name)

##### Data lengths check

In [347]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 8204 encrypted: 8823 decrypted: 7441
orig: 8204 encrypted: 8211 decrypted: 6706
orig: 8204 encrypted: 8902 decrypted: 7619
orig: 8204 encrypted: 8543 decrypted: 7084
orig: 8204 encrypted: 8471 decrypted: 6783
orig: 8204 encrypted: 8311 decrypted: 6311
orig: 8204 encrypted: 8902 decrypted: 7848
orig: 7576 encrypted: 7993 decrypted: 6702


### Szyfrowanie danych zaszyfrowanych

Czy obie metody są równoważne? - Nie!

##### Encryption

In [348]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

orig_data = []
encrypted_idat_chunks = []
for idat in idat_chunks:
    idat_data = idat[8:-4]

    orig_data.append(idat_data)
    encrypted_data = RSA.encryptData(idat_data, pubkey)

    idat_type = b'IDAT'
    new_len = len(encrypted_data).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_data))
    new_chunk = new_len + idat_type + encrypted_data + new_crc

    encrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
    
all_chunks = crit_chunks_no_data

timestamp = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
encrypt_img_path = writeEncrypted(all_chunks, png_name, timestamp)
keyfile = saveKeys(keypair, png_name, timestamp)

##### Decryption

In [349]:
keys = loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

decrypted_data = []
decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    chunk_data = chunk[8:-4]
    decrypted_chunk_data = RSA.decryptData(chunk_data, privkey)
    decrypted_data.append(decrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(decrypted_chunk_data).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_chunk_data))
    new_chunk = new_len + idat_type + decrypted_chunk_data + new_crc

    decrypted_idat_chunks.append(new_chunk)


crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
    
all_chunks = crit_chunks_no_data

writeDecrypted(all_chunks, png_name)

##### Data lengths check

In [350]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 7576 encrypted: 8215 decrypted: 7576


## Porównanie metod szyfrowania: ECB (Electronic CodeBook) oraz CBC (Cypher Clock Chaining)

##### Get new keypair

In [351]:
keypair = RSA.generateKeypair()

### ECB

#### Encryption

In [352]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

orig_data = []
encrypted_idat_chunks = []
for idat in idat_chunks:
    idat_data = idat[8:-4]

    orig_data.append(idat_data)
    encrypted_data = RSA.encryptECB(idat_data, pubkey)

    idat_type = b'IDAT'
    new_len = len(encrypted_data).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_data))
    new_chunk = new_len + idat_type + encrypted_data + new_crc

    encrypted_idat_chunks.append(new_chunk)
    
crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

timestamp = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
encrypt_img_path = writeEncrypted(all_chunks, png_name, timestamp)
keyfile = saveKeys(keypair, png_name, timestamp)


#### Decryption

In [353]:

keys = loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

decrypted_data = []
decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    chunk_data = chunk[8:-4]
    decrypted_chunk_data = RSA.decryptECB(chunk_data, privkey)
    decrypted_data.append(decrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(decrypted_chunk_data).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_chunk_data))
    new_chunk = new_len + idat_type + decrypted_chunk_data + new_crc

    decrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

writeDecrypted(all_chunks, png_name)

### CBC

#### Encryption

In [354]:
png = PNG(img_path)
png_name = os.path.basename(img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)
pubkey = keypair[0]

block_size = (pubkey[0].bit_length() + 7) // 8
iv = random.randbytes(block_size)

orig_data = []
encrypted_idat_chunks = []
for idat in idat_chunks:
    idat_data = idat[8:-4]

    orig_data.append(idat_data)
    encrypted_data = RSA.encryptCBC(idat_data, pubkey, iv)

    idat_type = b'IDAT'
    new_len = len(encrypted_data).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + encrypted_data))
    new_chunk = new_len + idat_type + encrypted_data + new_crc

    encrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in encrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

timestamp = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
encrypt_img_path = writeEncrypted(all_chunks, png_name, timestamp)
keyfile = saveKeys(keypair, png_name, timestamp)


#### Decryption

In [355]:

keys = loadKeys(keyfile)
privkey = keys[1]

png = PNG(encrypt_img_path)
png_name = os.path.basename(encrypt_img_path)
crit_chunks = png.getCriticalChunks()
anc_chunks = png.getAncillaryChunks()
encrypted_idat_chunks, crit_chunks_no_data = png.getDataChunks(crit_chunks)

decrypted_data = []
decrypted_idat_chunks = []
for chunk in encrypted_idat_chunks:
    chunk_data = chunk[8:-4]
    decrypted_chunk_data = RSA.decryptCBC(chunk_data, privkey, iv)
    decrypted_data.append(decrypted_uncompressed)

    idat_type = b'IDAT'
    new_len = len(decrypted_chunk_data).to_bytes(4, 'big')
    new_crc = struct.pack('>I', zlib.crc32(idat_type + decrypted_chunk_data))
    new_chunk = new_len + idat_type + decrypted_chunk_data + new_crc

    decrypted_idat_chunks.append(new_chunk)

crit_chunks_no_data.insert(0, png.png_signature)
for chunk in decrypted_idat_chunks + anc_chunks:
    crit_chunks_no_data.insert(-1, chunk)
all_chunks = crit_chunks_no_data

writeDecrypted(all_chunks, png_name)

##### Data lengths check

In [356]:
for i, j, k in zip(idat_chunks, encrypted_idat_chunks, decrypted_idat_chunks):
    print(f"orig: {len(i)} encrypted: {len(j)} decrypted: {len(k)}")

orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 8204 encrypted: 8891 decrypted: 8204
orig: 7576 encrypted: 8215 decrypted: 7576


## Porównanie z gotową implementacją RSA

In [ ]:
from Crypto.PublicKey import RSA as CryptoRSA
from Crypto.Cipher import PKCS1_OAEP
from classes.rsa import RSA

key = CryptoRSA.generate(2048)

n = key.n
e = key.e
d = key.d

pubkey = (n, e)
privkey = (n, d)

plaintext = b"Test"

ciphertext_my = RSA.encryptData(plaintext, pubkey)
decrypted_my = RSA.decryptData(ciphertext_my, privkey)
print("Odszyfrowane (Twoja implementacja):", decrypted_my)

cipher = PKCS1_OAEP.new(key.publickey())
ciphertext_lib = cipher.encrypt(plaintext)

decipher = PKCS1_OAEP.new(key)
decrypted_lib = decipher.decrypt(ciphertext_lib)


Odszyfrowane (Twoja implementacja): bytearray(b'Testowe dane do zaszyfrowania')
Odszyfrowane (PyCryptodome PKCS1_OAEP): b'Testowe dane do zaszyfrowania'
Czy odszyfrowane dane są identyczne? True


## Porównanie plików

In [ ]:
files = ['dice', 'test', 'plte']

for file in files:
    print(f"File: {file}.png")
    png = PNG(f"media/{file}.png")
    anc_chunks = png.getAncillaryChunks()
    crit_chunks = png.getCriticalChunks()
    print(f"Crit chunks: {len(crit_chunks)} Anc chunks: {len(anc_chunks)}")
    print("Crit chunk sizes: ")
    for chunk in crit_chunks:
        data_len = int.from_bytes(chunk[:4])
        chunk_type = chunk[4:8].decode('utf-8')
        print(f"{chunk_type}: {data_len}")


File: dice.png
Crit chunks: 3 Anc chunks: 0
Crit chunk sizes: 
IHDR: 13
IDAT: 179502
IEND: 0
File: test.png
Crit chunks: 10 Anc chunks: 8
Crit chunk sizes: 
IHDR: 13
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 8192
IDAT: 7564
IEND: 0
File: plte.png
Crit chunks: 5 Anc chunks: 6
Crit chunk sizes: 
IHDR: 13
PLTE: 768
IDAT: 8192
IDAT: 2484
IEND: 0
